# Restricted Boltzmann Machines (RBMs) in Ember ML

This notebook provides an introduction to Restricted Boltzmann Machines (RBMs) using the Ember ML framework. We'll cover:

1. What are RBMs and how do they work?
2. Creating and training an RBM
3. Using RBMs for feature learning
4. Visualizing RBM weights and reconstructions

RBMs are generative stochastic neural networks that can learn a probability distribution over their inputs. They consist of a visible layer and a hidden layer, with no connections between units within the same layer.

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt

# Import Ember ML components
from ember_ml.ops import set_backend
from ember_ml.nn import tensor
from ember_ml import ops
from ember_ml.models.rbm import RestrictedBoltzmannMachine, train_rbm

# Set a backend (choose 'numpy', 'torch', or 'mlx')
set_backend('numpy')
print(f"Using backend: {ops.get_backend()}")

# Set random seed for reproducibility
tensor.set_seed(42)

## 1. What are RBMs?

Restricted Boltzmann Machines (RBMs) are a type of generative stochastic artificial neural network that can learn a probability distribution over its inputs.

Key characteristics of RBMs:
- They have a visible layer (input) and a hidden layer
- Connections exist only between visible and hidden units (not within layers)
- They are energy-based models trained using contrastive divergence
- They can be used for dimensionality reduction, feature learning, and generative modeling

Let's create a simple diagram to visualize the RBM architecture:

In [ ]:
# Create a simple visualization of RBM architecture
def plot_rbm_architecture(visible_size=6, hidden_size=4):
    plt.figure(figsize=(10, 6))
    
    # Plot visible units
    visible_x = tensor.ones(visible_size) * 1
    visible_y = tensor.linspace(1, visible_size, visible_size)
    plt.scatter(visible_x, visible_y, s=500, c='blue', label='Visible Units')
    
    # Plot hidden units
    hidden_x = tensor.one.ones(hidden_size) * 4
    hidden_y = tensor.linspace(1.5, hidden_size+0.5, hidden_size)
    plt.scatter(hidden_x, hidden_y, s=500, c='red', label='Hidden Units')
    
    # Plot connections
    for i in range(visible_size):
        for j in range(hidden_size):
            plt.plot([1, 4], [visible_y[i], hidden_y[j]], 'k-', alpha=0.2)
    
    # Add labels
    for i in range(visible_size):
        plt.text(1, visible_y[i], f"v{i+1}", ha='center', va='center', color='white', fontsize=12)
    
    for i in range(hidden_size):
        plt.text(4, hidden_y[i], f"h{i+1}", ha='center', va='center', color='white', fontsize=12)
    
    plt.title('Restricted Boltzmann Machine Architecture', fontsize=14)
    plt.xlim(0, 5)
    plt.ylim(0, visible_size+1)
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, 0), ncol=2)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Display the RBM architecture
plot_rbm_architecture(visible_size=6, hidden_size=4)

## 2. Generate Simple Data for Training

Let's generate some simple data to train our RBM. We'll create patterns with correlations between features.

In [ ]:
# Generate simple data with correlations
def generate_simple_data(n_samples=1000, n_features=10):
    # Generate random data
    data = tensor.random_normal((n_samples, n_features), mean=0.0, stddev=1.0)
    data = tensor.to_numpy(data)
    
    # Add correlations
    data_tensor = tensor.convert_to_tensor(data)
    
    # Make feature 1 and 2 correlated
    data_tensor = tensor.index_update(
        data_tensor, 
        tensor.index[:, 1], 
        ops.add(data_tensor[:, 1], ops.multiply(data_tensor[:, 0], 0.8))
    )
    
    # Make feature 3 and 4 correlated
    data_tensor = tensor.index_update(
        data_tensor, 
        tensor.index[:, 3], 
        ops.add(data_tensor[:, 3], ops.multiply(data_tensor[:, 2], 0.8))
    )
    
    # Make feature 5 a function of features 0, 2
    data_tensor = tensor.index_update(
        data_tensor, 
        tensor.index[:, 5], 
        ops.add(ops.multiply(data_tensor[:, 0], 0.4), ops.multiply(data_tensor[:, 2], 0.4))
    )
    
    # Convert back to numpy
    data = tensor.to_numpy(data_tensor)
    
    # Scale to [0, 1] for binary RBM
    data = (data - data.min()) / (data.max() - data.min())
    
    return data

# Generate data
data = generate_simple_data(n_samples=1000, n_features=10)
print(f"Generated data shape: {data.shape}")

# Plot correlation matrix
plt.figure(figsize=(10, 8))
correlation_matrix = np.corrcoef(data.T)
plt.imshow(correlation_matrix, cmap='coolwarm', vmin=-1, vmax=1)
plt.colorbar(label='Correlation')
plt.title('Feature Correlation Matrix')
plt.xticks(range(10), [f'F{i}' for i in range(10)])
plt.yticks(range(10), [f'F{i}' for i in range(10)])

# Add correlation values
for i in range(10):
    for j in range(10):
        plt.text(j, i, f"{correlation_matrix[i, j]:.2f}", 
                 ha="center", va="center", color="white" if abs(correlation_matrix[i, j]) > 0.5 else "black")

plt.tight_layout()
plt.show()

## 3. Create and Train an RBM

Now let's create an RBM and train it on our data using Ember ML's implementation.

In [ ]:
# Split data into training and validation sets


n_samples = data.shape[0]
n_train = int(n_samples * 0.8)

train_data = data[:n_train]
val_data = data[n_train:]

print(f"Training set: {train_data.shape} samples")
print(f"Validation set: {val_data.shape} samples")

# Convert to EmberTensor
train_tensor = tensor.convert_to_tensor(train_data, dtype=tensor.float32, device='cpu')
val_tensor = tensor.convert_to_tensor(val_data, dtype=tensor.float32, device='cpu')

# Create a simple data generator for training
def data_generator(data_tensor, batch_size=32):
    n_samples = tensor.shape(data_tensor)[0]
    indices = list(range(n_samples))
    np.random.shuffle(indices)
    
    for i in range(0, n_samples, batch_size):
        batch_indices = indices[i:min(i + batch_size, n_samples)]
        batch_data = np.take(tensor.to_numpy(data_tensor), batch_indices, axis=0)
        yield tensor.convert_to_tensor(batch_data, dtype=tensor.float32, device='cpu')

# Create RBM
visible_size = data.shape[1]  # Number of features
hidden_size = 5               # Number of hidden units

print(f"Creating RBM with {visible_size} visible units and {hidden_size} hidden units")
rbm = RestrictedBoltzmannMachine(visible_size=visible_size, hidden_size=hidden_size, device='cpu')

# Train RBM
print("\nTraining RBM...")
import time
start_time = time.time()

# Create data generator
train_gen = data_generator(train_tensor, batch_size=32)

# Train the RBM
losses = train_rbm(
    rbm=rbm,
    data_generator=train_gen,
    epochs=50,
    k=1  # Number of Gibbs sampling steps
)

training_time = time.time() - start_time
print(f"Training completed in {training_time:.2f} seconds")

## 4. Visualize Training Progress

Let's visualize the training progress by plotting the loss over epochs.

In [ ]:
# Plot training loss
plt.figure(figsize=(10, 6))
plt.plot(losses, 'b-')
plt.title('RBM Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.tight_layout()
plt.show()

## 5. Visualize RBM Weights

The weights of an RBM represent the connections between visible and hidden units. Let's visualize these weights to understand what patterns the RBM has learned.

In [ ]:
# Get the weights
weights = tensor.to_numpy(rbm.weights.data)

# Plot the weights
plt.figure(figsize=(12, 8))
plt.imshow(weights, cmap='coolwarm', aspect='auto')
plt.colorbar(label='Weight Value')
plt.title('RBM Weights (Visible x Hidden)')
plt.xlabel('Hidden Units')
plt.ylabel('Visible Units')
plt.xticks(range(hidden_size), [f'H{i}' for i in range(hidden_size)])
plt.yticks(range(visible_size), [f'V{i}' for i in range(visible_size)])

# Add weight values
for i in range(visible_size):
    for j in range(hidden_size):
        plt.text(j, i, f"{weights[i, j]:.2f}", 
                 ha="center", va="center", 
                 color="white" if abs(weights[i, j]) > 0.5 * weights.max() else "black")

plt.tight_layout()
plt.show()

## 6. Analyze Hidden Unit Activations

Let's analyze what each hidden unit has learned by examining its activations for different input patterns.

In [ ]:
# Get hidden activations for validation data
hidden_probs, _ = rbm.visible_to_hidden(val_tensor)
hidden_activations = tensor.to_numpy(hidden_probs)

# Plot distribution of hidden unit activations
plt.figure(figsize=(12, 8))
for i in range(hidden_size):
    plt.subplot(hidden_size, 1, i+1)
    plt.hist(hidden_activations[:, i], bins=30, alpha=0.7)
    plt.title(f'Hidden Unit {i} Activation Distribution')
    plt.xlim(0, 1)
    if i == hidden_size - 1:
        plt.xlabel('Activation')
    plt.ylabel('Count')
    plt.grid(True)

plt.tight_layout()
plt.show()

# Compute correlation between hidden units
plt.figure(figsize=(8, 6))
hidden_corr = np.corrcoef(hidden_activations.T)
plt.imshow(hidden_corr, cmap='coolwarm', vmin=-1, vmax=1)
plt.colorbar(label='Correlation')
plt.title('Hidden Unit Correlations')
plt.xticks(range(hidden_size), [f'H{i}' for i in range(hidden_size)])
plt.yticks(range(hidden_size), [f'H{i}' for i in range(hidden_size)])

# Add correlation values
for i in range(hidden_size):
    for j in range(hidden_size):
        plt.text(j, i, f"{hidden_corr[i, j]:.2f}", 
                 ha="center", va="center", 
                 color="white" if abs(hidden_corr[i, j]) > 0.5 else "black")

plt.tight_layout()
plt.show()

## 7. Reconstruct Data

One way to evaluate an RBM is to see how well it can reconstruct the original data. Let's reconstruct some validation samples and compare them to the originals.

In [ ]:
# Select a few samples from validation set
n_samples_to_show = 5
sample_indices = ops.random_choice(len(val_data), n_samples_to_show, replace=False)
samples = val_tensor[sample_indices]

# Reconstruct samples
reconstructed = rbm.reconstruct(samples)
reconstructed_np = tensor.to_numpy(reconstructed)
samples_np = tensor.to_numpy(samples)

# Plot original vs reconstructed
plt.figure(figsize=(15, 10))
for i in range(n_samples_to_show):
    # Plot original
    plt.subplot(n_samples_to_show, 2, 2*i+1)
    plt.bar(range(visible_size), samples_np[i], color='blue', alpha=0.7)
    plt.title(f'Sample {i+1} - Original')
    plt.ylim(0, 1)
    plt.xticks(range(visible_size), [f'V{j}' for j in range(visible_size)], rotation=45)
    plt.grid(True)
    
    # Plot reconstruction
    plt.subplot(n_samples_to_show, 2, 2*i+2)
    plt.bar(range(visible_size), reconstructed_np[i], color='red', alpha=0.7)
    plt.title(f'Sample {i+1} - Reconstructed')
    plt.ylim(0, 1)
    plt.xticks(range(visible_size), [f'V{j}' for j in range(visible_size)], rotation=45)
    plt.grid(True)

plt.tight_layout()
plt.show()

# Calculate reconstruction error
mse = stats.mean((samples_np - reconstructed_np) ** 2)
print(f"Mean Squared Error: {mse:.4f}")

## 8. Generate New Samples

RBMs are generative models, which means they can generate new samples that follow the learned distribution. Let's generate some new samples from our trained RBM.

In [ ]:
# Generate new samples
n_generated = 5
generated_samples = rbm.sample(n_generated, num_gibbs_steps=1000)
generated_np = tensor.to_numpy(generated_samples)

# Plot generated samples
plt.figure(figsize=(15, 10))
for i in range(n_generated):
    plt.subplot(n_generated, 1, i+1)
    plt.bar(range(visible_size), generated_np[i], color='green', alpha=0.7)
    plt.title(f'Generated Sample {i+1}')
    plt.ylim(0, 1)
    plt.xticks(range(visible_size), [f'V{j}' for j in range(visible_size)])
    plt.grid(True)

plt.tight_layout()
plt.show()

## 9. Feature Learning with RBMs

RBMs can be used for feature learning and dimensionality reduction. Let's use the hidden layer activations as learned features and visualize them.

In [ ]:
# Get hidden activations for all data
all_data_tensor = tensor.convert_to_tensor(data, dtype=tensor.float32, device='cpu')
all_hidden_probs, _ = rbm.visible_to_hidden(all_data_tensor)
all_hidden_activations = tensor.to_numpy(all_hidden_probs)

# Plot 2D scatter plot of first two hidden units
plt.figure(figsize=(10, 8))
plt.scatter(all_hidden_activations[:, 0], all_hidden_activations[:, 1], alpha=0.5)
plt.title('Data Projected onto First Two Hidden Units')
plt.xlabel('Hidden Unit 0 Activation')
plt.ylabel('Hidden Unit 1 Activation')
plt.grid(True)
plt.tight_layout()
plt.show()

# If we have more than 2 hidden units, create a pairplot
if hidden_size > 2:
    plt.figure(figsize=(12, 10))
    for i in range(min(4, hidden_size)):
        for j in range(min(4, hidden_size)):
            plt.subplot(min(4, hidden_size), min(4, hidden_size), i*min(4, hidden_size) + j + 1)
            if i == j:
                plt.hist(all_hidden_activations[:, i], bins=20, alpha=0.7)
                plt.title(f'H{i}')
            else:
                plt.scatter(all_hidden_activations[:, j], all_hidden_activations[:, i], alpha=0.1, s=5)
                plt.xlabel(f'H{j}')
                plt.ylabel(f'H{i}')
    
    plt.tight_layout()
    plt.show()

## 10. Conclusion

In this notebook, we've explored the basics of Restricted Boltzmann Machines using Ember ML. We've covered:

1. The architecture and principles of RBMs
2. Training an RBM on synthetic data
3. Visualizing RBM weights and hidden unit activations
4. Reconstructing data and generating new samples
5. Using RBMs for feature learning

RBMs can be used for various tasks, including:
- Dimensionality reduction
- Feature learning
- Generative modeling
- Anomaly detection (as shown in the advanced anomaly detection notebook)
- Collaborative filtering and recommendation systems

Ember ML provides a flexible, backend-agnostic implementation of RBMs that can be used across different computational backends.